# 0. 패키지 임포트

In [19]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from decouple import config
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1. 기본 세팅

In [2]:
load_dotenv()

True

In [3]:
pc = Pinecone()

In [4]:
# embedding = OpenAIEmbeddings(model="text-embedding-3-large")
embedding = UpstageEmbeddings(model="solar-embedding-1-large")
index_name = config("PINECONE_INDEX")


In [5]:
database = PineconeVectorStore(
    embedding=embedding, 
    index_name=index_name
)

In [6]:
retriever = database.as_retriever(search_kwargs={"k": 10})
prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model="gpt-4.1-nano")

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs={"prompt": prompt},
)

# 2. 선다형 문제로 검증 실행

In [ ]:
df = pd.read_csv("법령문제_금융은행보험.csv")
df.sample()

In [9]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):
    sample = df.loc[i]
    question = """아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
보기가 A,B,C,D 라면 1,2,3,4 라고 생각하고 답변하시오.
오직 숫자만 출력하시오.
출력 예시: 3
"""
    question += sample['문제내용'] + '\n'

    for i in range(1,5):
        col = '보기' + str(i)
        question += sample[col] + '\n'

    message = qa_chain.invoke({"query":question})['result']
    if message == str(sample['정답']):
        success += 1
    else:
        fail += 1
    gpt.append(message)
    act.append( str(sample['정답']))

In [12]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 26
오답 갯수 : 24
정확도 : 0.52
챗봇에 대답:  ['3', '2', '2', '3', '3', '2', '2', '1', '3', '1', '4', '2', '2', '2', '3', '3', '4', '3', '3', '3', '2', '2', '2', '1', '2', '3', '3', '2', '3', '2', '3', '3', '3', '3', '2', '2', '1', '2', '3', '2', '2', '3', '2', '4', '2', '2', '3', '3', '3', '3']
실제 답    :  ['1', '3', '2', '3', '3', '4', '4', '1', '2', '2', '3', '2', '2', '2', '3', '2', '1', '2', '3', '2', '2', '2', '2', '1', '2', '2', '3', '1', '1', '2', '2', '2', '1', '2', '2', '2', '1', '3', '1', '2', '2', '3', '2', '1', '4', '2', '2', '3', '1', '3']


# 3. 단답형 문제로 검증 실행

In [41]:
df = pd.read_csv("단답형_문제.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,정답,해설
77,금융실명거래및비밀보장에관한법률부칙제7조의규정에의한소득세등의계산방법에관한규칙(총리령)...,단답형,상,금융실명거래및비밀보장에관한법률부칙 제7조의 규정에 의한 소득세 등을 원천징수하는 목...,납세자의 불편 해소,제1조에 명시된 목적이다.


In [43]:
results = []
SIMILARITY_THRESHOLD = 0.7

In [44]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):

    question = """아래 문제를 보고 짧고 간단하게 답을 단답식으로 제출하시오.
    좋은 답변들 예시 : ['대주주와의 거래', '지배구조법 대주주', '금융위원회', '제12조', '5%']
    '대통령령으로 정하는 기간은 3개월이다.' -> '3개월' 
    '개인금융채권의 원금이 5천만원 이상이다.'-> '5천만원'
    이와같이 사족없이 30자 이내로 핵심 단어의 단답형으로 답변하시오.
    """

    sample = df.loc[i]
    question += sample['문제내용']
    answer = sample['정답']


    
    message = qa_chain.invoke({"query": question})['result']

    ground_truth_embedding = embedding.embed_query(answer)
    prediction_embedding = embedding.embed_query(message)
    
    score = cosine_similarity(
        np.array(ground_truth_embedding).reshape(1, -1),
        np.array(prediction_embedding).reshape(1, -1)
    )[0][0]
    
    if score >= SIMILARITY_THRESHOLD:
        success += 1
    else:
        fail += 1
        
    gpt.append(message)
    act.append(answer)

In [45]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 20
오답 갯수 : 30
정확도 : 0.4
챗봇에 대답:  ['개인금융채권 정의', '대체로 정하는 사유 없음', '3개월', '5천만원', '추심 위탁 통지 방식은 별도 규정 없음', '개인금융채권은 채권금융회사 등이 금전대부, 양수, 할인 등으로 보유한 채권.', '채권금융회사등', '채무조정은 채무내용 변경', '추심은 채무자가 변제하지 않는 채권을 소재파악, 변제촉구, 수령행위하는 것.', '기한의 이익 상실예정일 통지 기한', '개인금융채권을 추심하는 자로서 채권금융회사, 채권추심회사, 위임직채권추심인, 대부업자 등.', '개인금융채권 관련 규정은 법률 제14조의2(과징금) 및 제7조(이자율 제한) 등에 명시되어 있다.', '개인금융채권이 채무조정이 끝났거나, 사망 여부가 미확정되었거나, 소송중인 경우, 또는 대통령령으로 정하는 권리 또는 상황일 때 양도하지 못한다.', '국제금융기구 출자규정', '자금중개회사의 인가', '출자금 단위 결정권자', '2019년 11월 26일', '02-2100-6114', '국제금융기구 출자금반영', '금융위원회 위원장', '법률 제16580호', '금융위원회', '금융거래지표법', '승인신청서  \n시험자료  \n서류첨부', '최소 5명', '중단사유 안내', '2024년 1월 2일', '제33조, 제8조', '금융거래지표법', '기초자료란 중요지표 산출에 이용되는 시장거래 자료', '2019', '중요지표 산출기관의 의무는 업무중단 신고', '- 지정 기한: 6개월  \r\n- 사유: 업종변경, 변경, 승인신청 등  \r\n- 법령: 대통령령으로 정하는 기준', '5조 1000억', '금융시장 안정, 건전경영, 소비자보호', '금융업 수행 회사', '2021년 6월 30일', '금융복합기업집단법', '지정 해제 사유: 요건 미충족', '내부거래: 대주주 상대 행위', '대주주와의 거래 제한', '금융위원회', '금융복합기업집단법 제12조', '5%', '“특수관계인”', '신용공여한도, 신고, 공시'